# pyLDAvis

[`pyLDAvis`](https://github.com/bmabey/pyLDAvis) is a python libarary for interactive topic model visualization.
It is a port of the fabulous [R package](https://github.com/cpsievert/LDAvis) by Carson Sievert and Kenny Shirley.  They did the hard work of crafting an effective visualization. `pyLDAvis` makes it easy to use the visualiziation from Python and, in particualr, IPython notebooks. To learn more about the method behind the visualization I suggest reading the [original paper](http://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) explaining it.

This notebook provides a quick overview of how to use `pyLDAvis`. Refer to the [documenation](https://pyldavis.readthedocs.org/en/latest/) for details.


## BYOM - Bring your own model

`pyLDAvis` is agnostic to how your model was trained. To visualize it you need to provide the topic-term distribtuions, document-topic distributions, and basic information about the corpus which the model was trained on. The main function is the [`prepare`](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.prepare) function that will transform your data into the format needed for the visualization.

Below we load a model trained in R and then visualize it. The model was trained on a corpus of 2000 movie reviews parsed by [Pang and Lee (ACL, 2004)](http://www.cs.cornell.edu/people/pabo/movie-review-data/), originally gathered from the IMDB archive of the rec.arts.movies.reviews newsgroup.

In [1]:
import json
import numpy as np

def load_R_model(filename):
    with open(filename, 'r') as j:
        data_input = json.load(j)
    data = {'topic_term_dists': data_input['phi'], 
            'doc_topic_dists': data_input['theta'],
            'doc_lengths': data_input['doc.length'],
            'vocab': data_input['vocab'],
            'term_frequency': data_input['term.frequency']}
    return data

movies_model_data = load_R_model('data/movie_reviews_input.json')

print('Topic-Term shape: %s' % str(np.array(movies_model_data['topic_term_dists']).shape))
print('Doc-Topic shape: %s' % str(np.array(movies_model_data['doc_topic_dists']).shape))

Topic-Term shape: (20, 14567)
Doc-Topic shape: (2000, 20)


Now that we have the data loaded we use the `prepare` function:

In [2]:
import pyLDAvis
movies_vis_data = pyLDAvis.prepare(**movies_model_data)

Once you have the visualization data prepared you can do a number of things with it. You can [save the vis](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.save_html) to an stand-alone HTML file, [serve it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.show), or [dispaly it](https://pyldavis.readthedocs.org/en/latest/modules/API.html#pyLDAvis.display) in the notebook. Let's go ahead and display it:

In [4]:
pyLDAvis.display(movies_vis_data)

Pretty, huh?! Again, you should be thanking the original [LDAvis people](https://github.com/cpsievert/LDAvis) for that. You may thank me for the IPython integartion though. :) Aside from being aesthetically pleasing this visualization more importantly represents a lot of information about the topic model that is hard to take in all at once with ad-hoc queries. To learn more about the visual elements and how they help you explore your model see [this documentation](http://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf) from the original R project and this presentation ([slides](https://speakerdeck.com/bmabey/visualizing-topic-models), [video](https://www.youtube.com/watch?v=tGxW2BzC_DU)).


To see other models visualzied check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Movie%20Reviews,%20AP%20News,%20and%20Jeopardy.ipynb).

*ProTip:* To avoid tediously typing in `display` all the time use:

In [5]:
pyLDAvis.enable_notebook()

By default the topics are projected to the 2D plane using [PCoA](https://en.wikipedia.org/wiki/PCoA) on a distance matrix created using the [Jensen-Shannon divergence](https://en.wikipedia.org/wiki/Jensen–Shannon_divergence) on the topic-term distritbuions. You can pass in a different multidimensional scaling function via the `mds` pararameter. In addition to `pcoa` other provided options are `tsne` and `mmds` which operate on the same JS-divergence distance matrix. Both `tsne` and `mmds` require that you have sklearn installed. Here is `tnse` in action:

In [6]:
pyLDAvis.prepare(mds='tsne', **movies_model_data)

PreparedData(topic_coordinates=            Freq  cluster  topics           x            y
topic                                                     
18     53.760681        1       1  194.893209   -52.310996
0       7.259356        1       2  193.997766  -184.542471
10      5.255364        1       3   -8.209169  1096.475595
16      3.465402        1       4   43.069640   177.229426
13      3.062695        1       5  -54.455539   -13.995972
14      2.541896        1       6   82.260655  -157.601554
12      2.491749        1       7  -83.501828   234.075783
9       2.420484        1       8 -157.662094    28.192397
4       2.172924        1       9  364.968006   -65.145731
6       2.081217        1      10  -92.370417   118.384782
19      1.930721        1      11   31.341434    62.068453
15      1.807655        1      12  -29.385923  -121.314826
17      1.765011        1      13  -21.323004 -1065.213530
8       1.687769        1      14  182.746291   177.804042
2       1.659404        1      15  158.083441    61.703529
7       1.480147        1      16   75.434268   -39.512479
1       1.407836        1      17 -416.931713   108.135299
11      1.361212        1      18 -239.154616   -50.075218
3       1.346567        1      19 -143.069555  -132.952029
5       1.041912        1      20  -18.067080  -241.760062, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
1     Default  5510.000000       movie  5510.000000  30.0000  30.0000
0     Default  8913.000000        film  8913.000000  29.0000  29.0000
2     Default  2399.000000        good  2399.000000  28.0000  28.0000
7     Default  1922.000000   character  1922.000000  27.0000  27.0000
4     Default  2130.000000       story  2130.000000  26.0000  26.0000
13    Default  1386.000000         bad  1386.000000  25.0000  25.0000
5     Default  2104.000000       films  2104.000000  24.0000  24.0000
9     Default  1565.000000        life  1565.000000  23.0000  23.0000
3     Default  2403.000000        time  2403.000000  22.0000  22.0000
10    Default  1492.000000        plot  1492.000000  21.0000  21.0000
6     Default  1943.000000  characters  1943.000000  20.0000  20.0000
23    Default  1100.000000        love  1100.000000  19.0000  19.0000
16    Default  1271.000000      scenes  1271.000000  18.0000  18.0000
18    Default  1203.000000        dont  1203.000000  17.0000  17.0000
20    Default  1150.000000      action  1150.000000  16.0000  16.0000
22    Default  1145.000000       great  1145.000000  15.0000  15.0000
14    Default  1388.000000       scene  1388.000000  14.0000  14.0000
11    Default  1427.000000      movies  1427.000000  13.0000  13.0000
21    Default  1143.000000         hes  1143.000000  12.0000  12.0000
12    Default  1423.000000      people  1423.000000  11.0000  11.0000
25    Default  1055.000000         big  1055.000000  10.0000  10.0000
68    Default   617.000000        high   617.000000   9.0000   9.0000
39    Default   826.000000       funny   826.000000   8.0000   8.0000
38    Default   824.000000      comedy   824.000000   7.0000   7.0000
53    Default   712.000000        show   712.000000   6.0000   6.0000
37    Default   850.000000      things   850.000000   5.0000   5.0000
45    Default   772.000000        john   772.000000   4.0000   4.0000
27    Default  1055.000000        back  1055.000000   3.0000   3.0000
15    Default  1322.000000         man  1322.000000   2.0000   2.0000
41    Default   805.000000       thing   805.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3126  Topic20    29.668611    duchovny    33.961442   4.4290  -5.2574
4424  Topic20    20.104262        glen    23.406589   4.4120  -5.6466
2263  Topic20    35.407220        cole    47.489129   4.2705  -5.0806
7040  Topic20    11.496348      briggs    12.848233   4.4529  -6.2055
2790  Topic20    27.755741       poker    38.839650   4.2281  -5.3241

Here is `mmds` in action:

In [12]:
pyLDAvis.prepare(mds='mmds', **movies_model_data)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     53.760681        1       1 -0.182340 -0.123314
0       7.259356        1       2 -0.443831  0.048992
10      5.255364        1       3 -0.073942 -0.183095
16      3.465402        1       4 -0.142915 -0.413945
13      3.062695        1       5  0.093895  0.394980
14      2.541896        1       6 -0.128830  0.179642
12      2.491749        1       7 -0.337237  0.194800
9       2.420484        1       8 -0.335648 -0.300205
4       2.172924        1       9  0.035768 -0.427823
6       2.081217        1      10  0.273396 -0.358201
19      1.930721        1      11  0.134881 -0.231274
15      1.807655        1      12  0.225152  0.343112
17      1.765011        1      13 -0.379842 -0.080420
8       1.687769        1      14  0.097243  0.107169
2       1.659404        1      15 -0.245204  0.366187
7       1.480147        1      16 -0.072955  0.406446
1       1.407836        1      17  0.296788 -0.133130
11      1.361212        1      18  0.439286 -0.104816
3       1.346567        1      19  0.386618  0.082781
5       1.041912        1      20  0.359718  0.232117, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
1     Default  5510.000000       movie  5510.000000  30.0000  30.0000
0     Default  8913.000000        film  8913.000000  29.0000  29.0000
2     Default  2399.000000        good  2399.000000  28.0000  28.0000
7     Default  1922.000000   character  1922.000000  27.0000  27.0000
4     Default  2130.000000       story  2130.000000  26.0000  26.0000
13    Default  1386.000000         bad  1386.000000  25.0000  25.0000
5     Default  2104.000000       films  2104.000000  24.0000  24.0000
9     Default  1565.000000        life  1565.000000  23.0000  23.0000
3     Default  2403.000000        time  2403.000000  22.0000  22.0000
10    Default  1492.000000        plot  1492.000000  21.0000  21.0000
6     Default  1943.000000  characters  1943.000000  20.0000  20.0000
23    Default  1100.000000        love  1100.000000  19.0000  19.0000
16    Default  1271.000000      scenes  1271.000000  18.0000  18.0000
18    Default  1203.000000        dont  1203.000000  17.0000  17.0000
20    Default  1150.000000      action  1150.000000  16.0000  16.0000
22    Default  1145.000000       great  1145.000000  15.0000  15.0000
14    Default  1388.000000       scene  1388.000000  14.0000  14.0000
11    Default  1427.000000      movies  1427.000000  13.0000  13.0000
21    Default  1143.000000         hes  1143.000000  12.0000  12.0000
12    Default  1423.000000      people  1423.000000  11.0000  11.0000
25    Default  1055.000000         big  1055.000000  10.0000  10.0000
68    Default   617.000000        high   617.000000   9.0000   9.0000
39    Default   826.000000       funny   826.000000   8.0000   8.0000
38    Default   824.000000      comedy   824.000000   7.0000   7.0000
53    Default   712.000000        show   712.000000   6.0000   6.0000
37    Default   850.000000      things   850.000000   5.0000   5.0000
45    Default   772.000000        john   772.000000   4.0000   4.0000
27    Default  1055.000000        back  1055.000000   3.0000   3.0000
15    Default  1322.000000         man  1322.000000   2.0000   2.0000
41    Default   805.000000       thing   805.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
3126  Topic20    29.668611    duchovny    33.961442   4.4290  -5.2574
4424  Topic20    20.104262        glen    23.406589   4.4120  -5.6466
2263  Topic20    35.407220        cole    47.489129   4.2705  -5.0806
7040  Topic20    11.496348      briggs    12.848233   4.4529  -6.2055
2790  Topic20    27.755741       poker    38.839650   4.2281  -5.3241
1958  Topic20    36.363655         meg    55.388825   4.1433  -5.0539
2990  Topic20    25.842871     malcolm 

## Making the common case easy - Gensim and others!

Built on top of the generic `prepare` function are helper functions for [gensim](https://radimrehurek.com/gensim/) and [GraphLab Create](https://dato.com/products/create/). To demonstrate below I am loading up a trained gensim model and coresponding dictionary and corpus (see [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/Gensim%20Newsgroup.ipynb) for how these were created):

In [9]:
import gensim

dictionary = gensim.corpora.Dictionary.load('newsgroups.dict')
corpus = gensim.corpora.MmCorpus('newsgroups.mm')
lda = gensim.models.ldamodel.LdaModel.load('newsgroups_50.model')

In the dark ages in order to inspect our topics all we had was `show_topics` and friends:

In [8]:
lda.show_topics()

['0.031*drive + 0.017*scsi + 0.014*disk + 0.012*hard + 0.011*armed + 0.011*speed + 0.010*drives + 0.009*bit + 0.009*data + 0.009*system',
 '0.026*insurance + 0.016*federal + 0.009*compound + 0.009*lunar + 0.008*black + 0.008*reported + 0.008*rates + 0.007*interests + 0.007*sea + 0.007*empire',
 '0.028*war + 0.019*arab + 0.017*land + 0.016*military + 0.016*russian + 0.014*population + 0.014*policy + 0.013*history + 0.009*states + 0.009*center',
 '0.025*coverage + 0.015*good + 0.014*mit + 0.012*morris + 0.012*cover + 0.010*tie + 0.010*new + 0.009*hallam + 0.009*rangers + 0.008*xlib',
 '0.016*palestinians + 0.012*win + 0.011*soldiers + 0.011*japanese + 0.011*republic + 0.010*dale + 0.010*libertarian + 0.010*democratic + 0.010*trade + 0.009*cultural',
 '0.041*new + 0.027*york + 0.026*san + 0.024*university + 0.024*los + 0.022*colorado + 0.022*angeles + 0.016*police + 0.015*california + 0.015*southern',
 "0.017*would + 0.013*don't + 0.010*one + 0.010*think + 0.010*like + 0.009*people + 0.00

Thankfully, in addition to these *still helpful functions*, we can get a feel for all of the topics with this one-liner:

In [11]:
import pyLDAvis.gensim

pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
46     12.857371        1       1  0.261719  0.065079
30      7.064369        1       2  0.223414  0.039362
6       6.561739        1       3  0.225668  0.119312
2       6.231526        1       4  0.115293 -0.064645
16      5.064460        1       5  0.188052  0.168255
48      4.565724        1       6 -0.201218  0.009103
39      4.343956        1       7  0.209195 -0.125400
28      3.028544        1       8  0.147132 -0.174707
47      2.920834        1       9  0.140357  0.135432
31      2.587286        1      10  0.137396 -0.073357
49      2.312800        1      11  0.016563 -0.005124
0       2.272079        1      12  0.131888  0.039923
7       2.047147        1      13  0.018617  0.186202
41      1.971782        1      14  0.121525 -0.023405
32      1.927232        1      15  0.051571 -0.233736
40      1.808287        1      16 -0.087086  0.018227
5       1.753615        1      17  0.084853  0.081672
20      1.717998        1      18  0.021080  0.032786
38      1.623964        1      19 -0.069950  0.098264
18      1.576322        1      20  0.001086 -0.118720
9       1.374833        1      21 -0.050045  0.164712
11      1.315061        1      22 -0.065465 -0.129421
35      1.294126        1      23  0.068778  0.117125
43      1.240337        1      24 -0.035228  0.136413
45      1.179658        1      25  0.088970 -0.024780
13      1.162380        1      26 -0.081018  0.100031
34      1.156061        1      27  0.091613 -0.187025
37      1.128807        1      28 -0.013147 -0.021178
21      1.103078        1      29  0.007740  0.071922
44      1.025090        1      30 -0.072332  0.065149
17      0.978828        1      31 -0.076000  0.054092
12      0.947931        1      32  0.043191 -0.165050
19      0.908854        1      33 -0.116769 -0.046389
15      0.900594        1      34 -0.028614  0.069812
26      0.839563        1      35 -0.062925  0.058828
27      0.833250        1      36  0.027215 -0.060409
4       0.819154        1      37 -0.058755  0.048751
24      0.818238        1      38  0.040752 -0.120218
23      0.746835        1      39 -0.138143 -0.005278
33      0.736402        1      40 -0.077180 -0.030381
10      0.681309        1      41 -0.040527 -0.035513
3       0.610906        1      42 -0.067085 -0.089631
36      0.609008        1      43 -0.114182 -0.037627
22      0.601290        1      44 -0.068577 -0.091707
25      0.574867        1      45 -0.127056 -0.007569
8       0.573916        1      46 -0.131165  0.003532
42      0.573233        1      47 -0.138972  0.022201
1       0.507317        1      48 -0.165909 -0.053731
29      0.454414        1      49 -0.188856  0.014065
14      0.067627        1      50 -0.187466  0.004755, topic_info=      Category          Freq                   Term         Total  loglift  \
term                                                                         
10285  Default  60497.000000                    'ax  60497.000000  30.0000   
17785  Default   4804.000000                    max   4804.000000  29.0000   
16749  Default   5509.000000                posting   5509.000000  28.0000   
210    Default   4917.000000                   host   4917.000000  27.0000   
2107   Default   4767.000000                   nntp   4767.000000  26.0000   
3996   Default   6249.000000                 people   6249.000000  25.0000   
13996  Default   3401.000000                    edu   3401.000000  24.0000   
1778   Default   5599.000000             university   5599.000000  23.0000   
13811  Default   7529.000000                article   7529.000000  22.0000   
4627   Default   2477.000000                israeli   2477.000000  21.0000   
19242  Default   2013.000000                  space   2013.000000  20.0000   
15469  Default   1676.000000                    key   1676.000000  19.0000   
1428   Default   2324.000000                    god   232

## GraphLab

As I mentioned above you can also easily visualize GraphLab TopicModels as well. Check out [this notebook](http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/GraphLab.ipynb#topic=7&lambda=0.41&term=) if you are interested in that.


## Go forth and visualize!

What are you waiting for? Go ahead and `pip install pyldavis`.